In [1]:
# Imports with all of my tools
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os
import random
from sympy import mod_inverse
from PIL import Image as im

%matplotlib inline


def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    print(image.shape)
    return image

In [2]:
#Set directory of my image of addie and retrieve it as a black and white image
#path = r"C:\Users\soule\OneDrive\Pictures\UI_Goku.jpg"
# path = r"C:\Users\soule\OneDrive\Pictures\Freddy.png"
path = r"C:\Users\soule\OneDrive\Pictures\addie.jpg"
l = get_image(path)
img = im.open(path)
colors = img.getpixel((0, 1))

#rows=450 and cols=600 based on our picture
rows, cols = l.shape
data = im.fromarray(l.astype(np.uint8))
data.show()

(410, 728)


In [3]:
def Generate_Point(a, b, p):
    x = random.randint(0, p)
    print(x)
    
    y = np.arange(0, p)
    possibilites = []
    for i in range(len(y)):
        yval = int((x ** 3) % p)
        yval = int((yval + (a * x)) % p)
        yval = int((yval + b) % p)
        testy = int((y[i] ** 2) % p)
        if testy == yval:
            possibilites.append(y[i])
            print(possibilites)
    if len(possibilites) == 0:
        return Generate_Point(a, b, p)
    else:
        yindex = random.randint(0, len(possibilites) - 1)
        return x, possibilites[yindex]

def IntegerDigits(p,b):
    arr = []
    while p>=1:
        arr.append(p%b)
        p = p//b
        print(arr)
    A = arr[::-1]
    return len(A)-1

def IntegerToArray(n,base):
    A = []
    while n>=1:
        A.append(n%base)
        n = n//base
    Aflip = A[::-1]
    return Aflip

def singleLargeInteger(A,base):
    integer = 0
    power = 0
    Anew = A[::-1]
    for i in range(len(Anew)):
        integer += (Anew[i] * base**i)
        #print(integer)
    return integer

def point_addition(x1, y1, x2, y2, p, a):
    z = 0
    if x1 == z and y1 == z:
        return x2, y2
    if x2 == z and y2 == z:
        return x1, y1
    if x1 == x2:
        if x1 == x2 and y1 == -y2 % p:
            return 0, 0
        else:
            den = int(mod_inverse(2 * y1, p) % p)
            m = int((3 * (x1 ** 2)) % p)
            m = int((m + a) % p)
            m = int((m * den) % p)
            m = int((((3 * (x1 ** 2)) % p + a) * (den)) % p)
            
    else:
        
        den = int(mod_inverse(x1 - x2, p) % p)
        m = int((y1 - y2) % p)
        m = int((m * den) % p)
        
    x = int((m ** 2) % p)
    x = int((x - x1 - x2) % p)
    
    y = int((x1 - x) % p)
    y = int((y * m) % p)
    y = int((y - y1) % p)
    
    return int(x), int(y)


def point_subtraction(x1, y1, x2, y2, p, a):
    y2 = int(-(y2))
    y2 = int(y2 % p)
    return point_addition(x1, y1, x2, y2, p, a)


def point_multiplication(x, y, k, p, a):
    xnew, ynew = int(x), int(y)
    for i in range(k - 1):
        xnew, ynew = point_addition(x, y, xnew, ynew, p, a)
        # print(xnew,ynew)
    return int(xnew), int(ynew)

In [4]:
#Create one large list
arr = []
for i in range(rows):
    for j in range(cols):
        arr.append(l[i][j])
print(arr[0], arr[1], arr[2])
print(len(arr))

50 50 50
298480


In [5]:
totalPixs = rows*cols
print(totalPixs)

groupSize = IntegerDigits(totalPixs,256)
print(groupSize)

groupedArray = np.reshape(arr, (totalPixs//groupSize, groupSize), order='C')
groupedArray = groupedArray.astype('int64')
print(groupedArray)

298480
[240]
[240, 141]
[240, 141, 4]
2
[[50 50]
 [50 51]
 [51 52]
 ...
 [32 32]
 [32 32]
 [32 32]]


In [6]:
Pm = []

for i in range(totalPixs//groupSize):
    newValue = singleLargeInteger(groupedArray[i],256)
    Pm.append(newValue)
    
print(len(Pm))

if len(Pm)%2 == 1:
    Pm.append(0)
    print('activated')
    
print(len(Pm))

149240
149240


In [7]:
#Pair every other value = Message points
imagePointsArray = np.reshape(Pm, (len(Pm) // 2, 2), order="C")
print(imagePointsArray)

[[12850 12851]
 [13108 13364]
 [13621 13621]
 ...
 [ 8994  8481]
 [ 8224  8224]
 [ 8224  8224]]


In [8]:
p = 1046527  # 16127
a = -5000
b = 76193

In [9]:
# armins private keys
private_A = 712
point_A = Generate_Point(a, b, p)
print(point_A)

492371
[414996]
[414996, 507313]
(492371, 507313)


In [10]:
# armins public key
public_A = point_multiplication(point_A[0], point_A[1], private_A, p, a)
print(public_A)

(444896, 897857)


In [11]:
# Erins key is 92
e_key = 92

In [12]:
coordinates, variables = imagePointsArray.shape
c2 = []
print(coordinates)
r = coordinates

c1 = point_multiplication(point_A[0], point_A[1], e_key, p, a)
added_to_M = point_multiplication(public_A[0], public_A[1], e_key, p, a)

for i in range(coordinates):
    
    c2_operation = point_addition(
        imagePointsArray[i][0],
        imagePointsArray[i][1],
        added_to_M[0],
        added_to_M[1],
        p,
        a,
    )
    c2.append(c2_operation[0])
    c2.append(c2_operation[1])
print(len(c2))

74620
149240


In [13]:
final_encrypted_array = []
for i in range(len(c2)):
    for j in range(groupSize):
        final_encrypted_array.append(c2[i]//(256**j))

In [14]:
encryption_arr = np.array(final_encrypted_array, dtype=int)
encryption_arr = np.reshape(encryption_arr, (rows, cols), order="C")
print(encryption_arr)
e2 = encryption_arr % 256
print(e2)

[[ 74951    292 561427 ...    707 118003    460]
 [884004   3453  50859 ...    707 118003    460]
 [123421    482 152847 ...    707 118003    460]
 ...
 [863860   3374  69925 ...   1107   6696     26]
 [353213   1379 682596 ...   3329 751157   2934]
 [353213   1379 682596 ...   3329 751157   2934]]
[[199  36  19 ... 195 243 204]
 [ 36 125 171 ... 195 243 204]
 [ 29 226  15 ... 195 243 204]
 ...
 [116  46  37 ...  83  40  26]
 [189  99 100 ...   1  53 118]
 [189  99 100 ...   1  53 118]]


In [15]:
print(459362 % 256)

98


In [16]:
e_data = im.fromarray(e2.astype(np.uint8))
e_data.show()

In [17]:
arr = []
for i in range(rows):
    for j in range(cols):
        arr.append(encryption_arr[i][j])
print(len(arr))
print(arr[0], arr[1], arr[2], arr[3])
# print(arr)

298480
74951 292 561427 2193


In [18]:
original_arr = []
arr = np.reshape(arr, (totalPixs//groupSize, groupSize), order="C")
for i in range(totalPixs//groupSize):
    original_arr.append(arr[i][0])
print(len(original_arr))

149240


x = x // 2

In [19]:
# newTotal = x * y
EmPointsArray = np.reshape(original_arr, (len(original_arr) // 2, 2), order="C")
print(EmPointsArray)

[[ 74951 561427]
 [829578 924902]
 [643926 873626]
 ...
 [768413 127570]
 [852236 751157]
 [852236 751157]]


In [20]:
coordinates, variables = EmPointsArray.shape
subtract_from_c2 = point_multiplication(c1[0],c1[1],private_A,p,a)
decrypted_array = []
print(coordinates)
r=coordinates
for i in range(coordinates):
    #C_E - (alpha * (G+B_1) + B_A)
    DP = point_subtraction(
        EmPointsArray[i][0],
        EmPointsArray[i][1],
        subtract_from_c2[0],
        subtract_from_c2[1],
        p,
        a
    )
    decrypted_array.append(DP[0])
    decrypted_array.append(DP[1])


74620


In [21]:
plaintext_array = []
#print(plaintext_array[0])
for i in range(len(decrypted_array)):
    arr = IntegerToArray(decrypted_array[i],256)
    while len(arr)>groupSize:
        arr.remove(arr[0])
    while len(arr)<groupSize:
        arr.insert(0,0)
    for j in arr:
        plaintext_array.append(j)
print(len(plaintext_array))

298480


In [22]:
decryption_arr = np.array(plaintext_array, dtype=int)
decryption_arr = np.reshape(decryption_arr, (rows, cols), order="C")
decryption_arr

array([[50, 50, 50, ...,  6,  6,  6],
       [51, 51, 51, ...,  6,  6,  6],
       [52, 52, 52, ...,  6,  6,  6],
       ...,
       [71, 71, 71, ..., 34, 34, 34],
       [71, 72, 72, ..., 32, 32, 32],
       [71, 72, 72, ..., 32, 32, 32]])

In [23]:
d_data = im.fromarray(decryption_arr.astype(np.uint8))
d_data.show()